In [1]:
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==0.6.0"
%matplotlib inline

In [3]:
!pip install import-ipynb

In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
# loading packages
import random
import logging
import os
import sys
import glob
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, datasets

from monai.utils import first
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.transforms import (Activations,
                              AddChanneld, 
                              AsDiscrete, 
                              Compose, 
                              LoadImaged, 
                              RandRotate90d, 
                              Resized, 
                              ScaleIntensityd,
                              RandAffined,
                              RandRotated,
                              RandGaussianNoised,
                              EnsureTyped,
                              EnsureType,
                              ToTensord)

from torch.utils.tensorboard import SummaryWriter
# import datetime
# from tensorflow import summary
# import tensorflow as tf
from monai.metrics import get_confusion_matrix, ConfusionMatrixMetric
import time
import import_ipynb
torch.cuda.empty_cache()


In [4]:
%cd '/gdrive/MyDrive/Colab Notebooks'

/gdrive/MyDrive/Colab Notebooks


In [26]:
def transfoms_augmentation():
  ''' transform and augment images
  return:
         transformed and augmentated images'''
  data_transforms = {
        'train':    Compose([
                     LoadImaged(keys = ["image"]),
                     AddChanneld(keys = ["image"]),
                     ScaleIntensityd(keys = ["image"]),
                     Resized(keys = ["image"], spatial_size = (128, 128, 128)),
                     RandAffined(keys = ["image"], prob=0.5, translate_range = 10),
                     RandRotate90d(keys=["image"], prob=0.8, spatial_axes=[0, 2]),
                     EnsureTyped(keys=["image"]),
                    #  ToTensord(keys = ["image"])
                     ]),
        'val':      Compose([
                     LoadImaged(keys=["image"]),
                     AddChanneld(keys=["image"]),
                     ScaleIntensityd(keys=["image"]),
                     Resized(keys=["image"], spatial_size=(128, 128, 128)),
                     EnsureTyped(keys=["image"]),
                    #  ToTensord(keys = ["image"])
                      ])
                     }
  return data_transforms

In [24]:
import data_preparation


In [7]:
data_path = data_preparation.data_path
data_path1 = data_preparation.data_path1

In [8]:
#prepare and load all training and validation data
training_data, validation_data, data = data_preparation.ready_all_data(data_path, data_path1) 

Both folders have now equal length
Both folders have now equal length


In [9]:
#length of dataset
print(f'length of training data: {len(training_data)}')
print(f'length of validation data: {len(validation_data)}')
print(f'length of total data: {len(data)}')

length of training data: 966
length of validation data: 242
length of total data: 1208


In [10]:
#inspecting data
train_index = np.random.randint(len(training_data))
print(f"training file at {train_index}:, image: {training_data[train_index]['image']} , label: {training_data[train_index]['label']}")
val_index = np.random.randint(len(validation_data))
print(f"validation file at {val_index}:, image: {validation_data[val_index]['image']}, label: {validation_data[val_index]['label']}")

training file at 530:, image: /gdrive/MyDrive/Medical Dataset/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_654_1/BraTS19_TCIA13_654_1_t1ce.nii.gz , label: 0
validation file at 67:, image: /gdrive/MyDrive/Medical Dataset/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA02_491_1/BraTS19_TCIA02_491_1_flair.nii.gz, label: 1


In [11]:
#checking files in dataset
LGG_count, HGG_count, count_flair, count_t1, count_t2, count_t1ce = data_preparation.count_entries(validation_data)
print(f'counting differnting classes entry in validation data with seed:{data_preparation.seed1}')
print(f'number of flairs: {count_flair}')
print(f'number of t1: {count_t1}')
print(f'number of t2: {count_t2}')
print(f'number of t1ce: {count_t1ce}')
print(f'LGG count: {LGG_count}')
print(f'HGG count: {HGG_count}')

counting differnting classes entry in validation data with seed:222
number of flairs: 61
number of t1: 55
number of t2: 67
number of t1ce: 59
LGG count: 121
HGG count: 121


In [12]:
LGG_count, HGG_count, count_flair, count_t1, count_t2, count_t1ce = data_preparation.count_entries(training_data)
print(f'counting differnting classes entry in training data with seed:{data_preparation.seed}')
print(f'number of flairs: {count_flair}')
print(f'number of t1: {count_t1}')
print(f'number of t2: {count_t2}')
print(f'number of t1ce: {count_t1ce}')
print(f'LGG count: {LGG_count}')
print(f'HGG count: {HGG_count}')

counting differnting classes entry in training data with seed:111
number of flairs: 241
number of t1: 247
number of t2: 235
number of t1ce: 243
LGG count: 483
HGG count: 483


In [13]:
post_pred = Compose([EnsureType(), Activations(softmax=True)])
post_label = Compose([EnsureType(), AsDiscrete(to_onehot=2)])

In [27]:
data_transforms = transforms_augmentation()
training_transforms = data_transforms['train']
validation_transforms = data_transforms['val']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

TypeError: ignored

In [15]:
check_ds = monai.data.Dataset(data=training_data, transform=training_transforms)
check_loader = DataLoader(check_ds, batch_size=8, num_workers=1, pin_memory=torch.cuda.is_available())
check_data = monai.utils.misc.first(check_loader)
print(check_data["image"].shape, check_data["label"].shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([8, 1, 128, 128, 128]) torch.Size([8])


In [83]:
# create a training data loader
train_ds = monai.data.Dataset(data=training_data, transform=training_transforms)
train_loader = DataLoader(train_ds, batch_size=6, shuffle=True, num_workers=1, pin_memory=torch.cuda.is_available(), drop_last=False)

    # create a validation data loader
val_ds = monai.data.Dataset(data=validation_data, transform=validation_transforms)
val_loader = DataLoader(val_ds, batch_size=6, num_workers=1, pin_memory=torch.cuda.is_available(), drop_last=False)

In [84]:
def train(model, optimizer, loss_fn, train_loader, val_loader, auc_metric, num_epochs=50):
  '''train the model on brain tumour dataset
  model: a deep leanring model
  optimizer: select a optimizer eg. Adam
  loss_fn: loss function 
  train_loader: load training data in batches
  val_loader: load validation data in batches
  '''
  val_interval = 2
  best_metric = -1
  best_metric_epoch = -1
  writer = SummaryWriter()
  for epoch in range(num_epochs):
    print('-'*45)
    epoch_loss = 0
    num_corrects = 0
    step = 0
    print(f'epoch: {epoch+1}/{num_epochs}')
    model.train()
    for data in train_loader:
      images, labels = data["image"].to(device), data["label"].to(device)
      outputs = model(images)
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      prediction = torch.argmax(outputs, dim=1)
      num_corrects += torch.eq(prediction, labels).sum().float()
      epoch_loss += loss.item()
      epoch_len = len(train_ds) // train_loader.batch_size
      print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
      writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)
    epoch_loss/=step
    epoch_acc = num_corrects/(len(train_ds)-(len(train_ds) - epoch_len * train_loader.batch_size))
    print(f'epoch training accuracy: {epoch_acc}', end= " ")
    print(f" epoch {epoch + 1} average loss: {epoch_loss:.4f}")
    if (epoch + 1) % val_interval == 0:
      model.eval()
      with torch.no_grad():
        y_pred = torch.tensor([], dtype=torch.float32, device=device)
        y = torch.tensor([], dtype=torch.long, device=device)
        for val_data in val_loader:
          val_images, val_labels = val_data["image"].to(device), val_data["label"].to(device)
          y_pred = torch.cat([y_pred, model(val_images)], dim=0)
          y = torch.cat([y, val_labels], dim=0)
        acc_value = torch.eq(y_pred.argmax(dim=1), y)
        acc_metric = acc_value.sum().item() / len(acc_value)
        y_onehot = [post_label(i) for i in decollate_batch(y)]
        y_pred_act = [post_pred(i) for i in decollate_batch(y_pred)]
        auc_metric(y_pred_act, y_onehot)
        auc_result = auc_metric.aggregate()
        auc_metric.reset()
        del y_pred_act, y_onehot
        if acc_metric > best_metric:
          best_metric = acc_metric
          best_metric_epoch = epoch + 1
          torch.save(model.state_dict(), "best_metric_model_classification3d_dict.pth")
          print("saved new best metric model")
        print("current epoch: {} current accuracy: {:.4f} current AUC: {:.4f} best accuracy: {:.4f} at epoch {}".format(
                        epoch + 1, acc_metric, auc_result, best_metric, best_metric_epoch))
        writer.add_scalar("val_accuracy", acc_metric, epoch + 1)
  print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
  writer.close()

In [85]:

model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
auc_metric = ROCAUCMetric()

In [1]:
train(model, optimizer, loss_function, train_loader, val_loader, auc_metric, 10)

NameError: ignored